# Preprocessing Data Komentar YouTube

Notebook ini melakukan preprocessing data komentar dengan langkah-langkah:
1. Filter Konteks (Strategi KETAT)
2. Bersihkan Teks (URL, Unicode, Emoji, lowercase, tanda baca, angka)
3. Tokenization
4. Hapus Stopwords
5. Gabungkan Kembali

In [1]:
# Import libraries
import pandas as pd
import re
import unicodedata
import demoji

# Download emoji data (run once)
demoji.download_codes()

C:\Users\rhzain\AppData\Local\Temp\ipykernel_29372\1846252205.py:8: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [2]:
# Load data
df = pd.read_csv('raw-scrape-yt.csv')
print(f"Total komentar: {len(df)}")
print("\nContoh data awal:")
df.head()

Total komentar: 2196

Contoh data awal:


,Video_ID,Teks_Komentar
0,MIo4tGN11j0,"berdoa tidak ada comment judol, amin."
1,MIo4tGN11j0,kereen.
2,MIo4tGN11j0,"Sempat mikir mau pindah ke negara sebelah, nge..."
3,MIo4tGN11j0,"Kalo kabur mau kemana ke Singapur ,emang di Si..."
4,MIo4tGN11j0,Jg ke malaysia.....pergi sono india bangla nep...


## 1. Filter Konteks

Membuang komentar noise yang tidak relevan

In [3]:
# --- FILTER 1: FILTER NOISE (Kode Anda, sudah bagus) ---
def filter_noise(teks):
    """
    Return True jika komentar VALID (BUKAN noise).
    Return False jika komentar adalah NOISE (spam, pendek, dll).
    """
    if pd.isna(teks) or len(teks.strip()) < 3:
        return False # Ini adalah noise
    
    teks_lower = teks.lower().strip()
    
    # Daftar pola noise yang harus dibuang
    noise_patterns = [
        r'^[@#]',  # Dimulai dengan @ atau #
        r'^\d+$',  # Hanya angka
        r'^[^\w\s]+$',  # Hanya simbol/emoji
        r'(subscribe|subs|subrek|gw klik subs)',  # Promosi subscribe
        r'(like|liek|liak|laik)\s*(dulu|dong|yuk)',  # Ajakan like
        r'(pin|ping|pin dong|pinn)',  # Minta pin
        r'^(wkwk|haha|xixi|kwkw|wkwkwk|hahaha)+$',  # Hanya ketawa
        r'^(amin|amiin|aamiin)+$',  # Hanya amin
        r'^(pertamax|first|kedua|ketiga)',  # Klaim urutan
        r'(giveaway|give away|kontes)',  # Promo
        r'^(yang \d{4})',  # "yang 2024", "yang 2025" (biasanya noise)
        r'(judol|judi online)', # Filter judi online
    ]
    
    for pattern in noise_patterns:
        if re.search(pattern, teks_lower):
            return False # Ini adalah noise
    
    # Komentar terlalu pendek (< 10 karakter)
    if len(teks_lower) < 10:
        return False # Ini adalah noise
    
    return True # Jika lolos semua = BUKAN noise

# --- FILTER 2: FILTER KONTEKS (Versi LONGGAR yang Efisien) ---
KEYWORD = [
    'kabur', 'pindah negara', 'leave indo', 'pajak', 'gaji', 'umr', 
    'biaya hidup', 'korupsi', 'sandwich', 'luar negeri', 
    'paspor', 'warga negara', 'singapur', 'singapore', 'australia', 'aussie', 'jepang', 'eropa', 'wni'
    'capek', 'lelah', 'pemerintah', 'mending', 'percuma', 'beban', 
    'suram', 'males', 'ga jelas', 'ga ada harapan', 'nyicil', 'susah',
    'setuju', 'sulit', 'stres', 'politik', 'birokrasi', 'konoha'
]

print(f"Total keyword unik untuk filter konteks: {len(KEYWORD)}")

def filter_konteks(teks):
    """
    Return True jika komentar RELEVAN (Strategi LONGGAR dan Efisien).
    Return False jika di luar konteks.
    """
    teks_lower = str(teks).lower()
    
    # Cukup cek satu kali terhadap list gabungan
    if any(kata in teks_lower for kata in KEYWORD):
        return True # Lolos jika mengandung SALAH SATU
        
    return False

# --- FUNGSI GABUNGAN (YANG HARUS DITERAPKAN) ---
def filter_final(teks):
    # 1. Cek apakah ini NOISE?
    if not filter_noise(teks):
        return False # Jika "wkwkwk", buang.
    
    # 2. Jika bukan noise, cek KONTEKS?
    if not filter_konteks(teks): # <- Memanggil fungsi baru yang efisien
        return False # Jika "resep nasi goreng", buang.
        
    # Hanya lolos jika BUKAN noise DAN RELEVAN
    return True

# --- Terapkan Filter ---
# Ganti 'TSeks_Komentar' dengan nama kolom Anda yang berisi teks mentah
df['is_valid'] = df['Teks_Komentar'].apply(filter_final)

df_filtered = df[df['is_valid']].copy()
df_filtered = df_filtered.drop('is_valid', axis=1)

print(f"Komentar sebelum filter: {len(df)}")
print(f"Komentar setelah filter: {len(df_filtered)}")
print(f"Komentar terbuang: {len(df) - len(df_filtered)}")

Total keyword unik untuk filter konteks: 36
Komentar sebelum filter: 2196
Komentar setelah filter: 641
Komentar terbuang: 1555


## 2. Preprocessing Teks

Melakukan preprocessing data dengan 5 tahap terpisah yang modular

In [4]:
import pandas as pd
import re
import unicodedata
import demoji
from nltk.corpus import stopwords

# --- 1. Inisialisasi Alat (Sekali saja) ---
print("="*60)
print("INISIALISASI PREPROCESSING TOOLS")
print("="*60)

# Impor Stemmer dari Sastrawi
try:
    from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
    print("✅ Sastrawi Stemmer berhasil dimuat")
    factory = StemmerFactory()
    stemmer_sastrawi = factory.create_stemmer()
except ImportError:
    print("❌ ERROR: Library Sastrawi tidak ditemukan.")
    print("   Silakan instal dengan: pip install Sastrawi")
    raise

# --- 2. Buat Daftar Stopwords Gabungan ---
stop_words_indo = set(stopwords.words('indonesian'))
stop_words_eng = set(stopwords.words('english'))
custom_stopwords_gaul = {
    'yg', 'dg', 'rt', 'dgn', 'ny', 'd', 'klo', 'kalo', 'amp', 'biar', 'bkn', 'na', 
    'nya', 'nih', 'sih', 'si', 'tau', 'tuh', 'utk', 'ya', 'ga', 'gak', 'gaes',
    'bang', 'bro', 'sob', 'gw', 'gua', 'lu', 'lo', 'wkwk', 'haha', 'wkwkwk', 
    'amin', 'amiin', 'aamiin', 'yuk', 'dong', 'deh', 'kok', 'sih'
}

# Gabungkan semua stopwords
stop_words_final = stop_words_indo.union(stop_words_eng, custom_stopwords_gaul)

print(f"✅ Stopwords berhasil dimuat:")
print(f"   - Indonesia: {len(stop_words_indo)} kata")
print(f"   - Inggris  : {len(stop_words_eng)} kata")
print(f"   - Gaul     : {len(custom_stopwords_gaul)} kata")
print(f"   - TOTAL    : {len(stop_words_final)} kata")
print("\n" + "="*60)


# --- 3. Definisi Fungsi Preprocessing ---

def tahap1_cleaning(teks):
    """
    Tahap 1: Membersihkan teks mentah dari URL, HTML, emoji, 
    tanda baca, dan angka.
    """
    if pd.isna(teks):
        return ""
    
    teks = str(teks)
    
    # Hapus URL
    teks = re.sub(r'http\S+|www\.\S+', '', teks)
    
    # Hapus HTML tags
    teks = re.sub(r'<.*?>', '', teks)
    
    # Ganti Emoji ke deskripsi
    teks = demoji.replace_with_desc(teks, sep=" ")
    
    # Normalisasi Unicode (font aneh)
    teks = unicodedata.normalize('NFKD', teks)
    
    # Lowercase
    teks = teks.lower()
    
    # Normalisasi kata berulang (capeeeek -> capek)
    teks = re.sub(r'(.)\1{2,}', r'\1', teks)
    
    # Hapus tanda baca dan angka (hanya sisakan huruf dan spasi)
    teks = re.sub(r'[^a-z\s]', ' ', teks)
    
    # Hapus spasi berlebih
    teks = re.sub(r'\s+', ' ', teks).strip()
    
    return teks


def tahap2_tokenisasi(teks):
    """
    Tahap 2: Tokenisasi - Memecah teks menjadi list kata-kata.
    """
    if not teks or pd.isna(teks):
        return []
    
    # Pecah berdasarkan spasi
    tokens = teks.split()
    
    # Filter kata yang terlalu pendek (< 3 karakter)
    tokens = [kata for kata in tokens if len(kata) >= 3]
    
    return tokens


def tahap3_stopword_removal(tokens):
    """
    Tahap 3: Hapus Stopwords dari list tokens.
    """
    if not tokens:
        return []
    
    # Hapus stopwords
    tokens_bersih = [kata for kata in tokens if kata not in stop_words_final]
    
    return tokens_bersih


def tahap4_stemming(tokens):
    """
    Tahap 4: Stemming - Mengubah kata ke bentuk dasar.
    (Ini proses LAMBAT, dilakukan per-kata)
    """
    if not tokens:
        return []
    
    # Stem setiap kata
    tokens_stemmed = [stemmer_sastrawi.stem(kata) for kata in tokens]
    
    return tokens_stemmed


def tahap5_gabung_kembali(tokens):
    """
    Tahap 5: Gabungkan tokens menjadi teks final.
    """
    if not tokens:
        return ""
    
    return " ".join(tokens)

print("✅ Semua fungsi preprocessing sudah siap!")

INISIALISASI PREPROCESSING TOOLS
✅ Sastrawi Stemmer berhasil dimuat
✅ Stopwords berhasil dimuat:
   - Indonesia: 757 kata
   - Inggris  : 198 kata
   - Gaul     : 40 kata
   - TOTAL    : 992 kata

✅ Semua fungsi preprocessing sudah siap!


### Tahap 1: Cleaning

Membersihkan teks dari URL, HTML, emoji, tanda baca, dan angka

In [5]:
print("="*60)
print("📝 TAHAP 1/5: CLEANING")
print("="*60)
print("Membersihkan URL, HTML, emoji, tanda baca, dan angka...")

df_filtered['teks_tahap1'] = df_filtered['Teks_Komentar'].apply(tahap1_cleaning)

print("✅ Selesai!")
print(f"\nContoh hasil cleaning:")
print(f"ASLI  : {df_filtered['Teks_Komentar'].iloc[0][:70]}...")
print(f"CLEAN : {df_filtered['teks_tahap1'].iloc[0][:70]}...")
print("="*60)

📝 TAHAP 1/5: CLEANING
Membersihkan URL, HTML, emoji, tanda baca, dan angka...
✅ Selesai!

Contoh hasil cleaning:
ASLI  : Kalo kabur mau kemana ke Singapur ,emang di Singapur tinggal dimana ra...
CLEAN : kalo kabur mau kemana ke singapur emang di singapur tinggal dimana rak...
✅ Selesai!

Contoh hasil cleaning:
ASLI  : Kalo kabur mau kemana ke Singapur ,emang di Singapur tinggal dimana ra...
CLEAN : kalo kabur mau kemana ke singapur emang di singapur tinggal dimana rak...


### Tahap 2: Tokenisasi

Memecah teks menjadi list kata-kata dan filter kata pendek

In [6]:
print("="*60)
print("📝 TAHAP 2/5: TOKENISASI")
print("="*60)
print("Memecah teks menjadi kata-kata...")

df_filtered['tokens_tahap2'] = df_filtered['teks_tahap1'].apply(tahap2_tokenisasi)

print("✅ Selesai!")
print(f"\nContoh hasil tokenisasi:")
tokens_contoh = df_filtered['tokens_tahap2'].iloc[0]
print(f"TEKS   : {df_filtered['teks_tahap1'].iloc[0][:60]}...")
print(f"TOKENS : {tokens_contoh[:8]}...")
print(f"JUMLAH : {len(tokens_contoh)} kata")
print("="*60)

📝 TAHAP 2/5: TOKENISASI
Memecah teks menjadi kata-kata...
✅ Selesai!

Contoh hasil tokenisasi:
TEKS   : kalo kabur mau kemana ke singapur emang di singapur tinggal ...
TOKENS : ['kalo', 'kabur', 'mau', 'kemana', 'singapur', 'emang', 'singapur', 'tinggal']...
JUMLAH : 26 kata


### Tahap 3: Stopword Removal

Menghapus stopwords (Indonesia, Inggris, dan kata gaul)

In [7]:
print("="*60)
print("📝 TAHAP 3/5: STOPWORD REMOVAL")
print("="*60)
print("Menghapus stopwords...")

df_filtered['tokens_tahap3'] = df_filtered['tokens_tahap2'].apply(tahap3_stopword_removal)

print("✅ Selesai!")
print(f"\nContoh hasil stopword removal:")
tokens_before = df_filtered['tokens_tahap2'].iloc[0]
tokens_after = df_filtered['tokens_tahap3'].iloc[0]
print(f"SEBELUM : {tokens_before[:8]}...")
print(f"SESUDAH : {tokens_after[:8]}...")
print(f"JUMLAH  : {len(tokens_before)} kata → {len(tokens_after)} kata")
print(f"REDUKSI : {len(tokens_before) - len(tokens_after)} kata dihapus")
print("="*60)

📝 TAHAP 3/5: STOPWORD REMOVAL
Menghapus stopwords...
✅ Selesai!

Contoh hasil stopword removal:
SEBELUM : ['kalo', 'kabur', 'mau', 'kemana', 'singapur', 'emang', 'singapur', 'tinggal']...
SESUDAH : ['kabur', 'kemana', 'singapur', 'emang', 'singapur', 'tinggal', 'dimana', 'rakyat']...
JUMLAH  : 26 kata → 19 kata
REDUKSI : 7 kata dihapus


### Tahap 4: Stemming

Mengubah kata ke bentuk dasar (proses lambat)

In [8]:
print("="*60)
print("📝 TAHAP 4/5: STEMMING")
print("="*60)
print("Mengubah kata ke bentuk dasar...")
print("⚠️  Proses ini lambat, harap sabar...")

df_filtered['tokens_tahap4'] = df_filtered['tokens_tahap3'].apply(tahap4_stemming)

print("✅ Selesai!")
print(f"\nContoh hasil stemming:")
tokens_before = df_filtered['tokens_tahap3'].iloc[0]
tokens_after = df_filtered['tokens_tahap4'].iloc[0]
print(f"SEBELUM : {tokens_before[:5]}")
print(f"SESUDAH : {tokens_after[:5]}")
print("="*60)

📝 TAHAP 4/5: STEMMING
Mengubah kata ke bentuk dasar...
⚠️  Proses ini lambat, harap sabar...
✅ Selesai!

Contoh hasil stemming:
SEBELUM : ['kabur', 'kemana', 'singapur', 'emang', 'singapur']
SESUDAH : ['kabur', 'mana', 'singapur', 'emang', 'singapur']
✅ Selesai!

Contoh hasil stemming:
SEBELUM : ['kabur', 'kemana', 'singapur', 'emang', 'singapur']
SESUDAH : ['kabur', 'mana', 'singapur', 'emang', 'singapur']


### Tahap 5: Gabung Kembali & Simpan

Menggabungkan tokens menjadi teks final dan menyimpan hasil

In [9]:
print("="*60)
print("📝 TAHAP 5/5: GABUNG KEMBALI & SIMPAN")
print("="*60)
print("Menggabungkan tokens menjadi teks final...")

df_filtered['teks_final'] = df_filtered['tokens_tahap4'].apply(tahap5_gabung_kembali)

# Buang baris dengan teks kosong setelah preprocessing
df_final = df_filtered[df_filtered['teks_final'].str.strip() != ''].copy()

print("✅ Selesai!")
print(f"\n📊 Ringkasan:")
print(f"   Total komentar valid   : {len(df_final):,}")
print(f"   Komentar kosong terbuang : {len(df_filtered) - len(df_final):,}")

# Simpan hasil akhir
output_file = 'data_preprocessed.csv'
df_final[['Teks_Komentar', 'teks_final']].to_csv(output_file, index=False, encoding='utf-8')
print(f"\n💾 Data berhasil disimpan ke '{output_file}'")

# Tampilkan contoh hasil
print(f"\n📋 Contoh hasil akhir:")
print(f"   ASLI  : {df_final['Teks_Komentar'].iloc[0][:60]}...")
print(f"   FINAL : {df_final['teks_final'].iloc[0][:60]}...")

print("\n" + "="*60)
print("🎉 PREPROCESSING SELESAI!")
print("="*60)

📝 TAHAP 5/5: GABUNG KEMBALI & SIMPAN
Menggabungkan tokens menjadi teks final...
✅ Selesai!

📊 Ringkasan:
   Total komentar valid   : 641
   Komentar kosong terbuang : 0

💾 Data berhasil disimpan ke 'data_preprocessed.csv'

📋 Contoh hasil akhir:
   ASLI  : Kalo kabur mau kemana ke Singapur ,emang di Singapur tinggal...
   FINAL : kabur mana singapur emang singapur tinggal mana rakyat singa...

🎉 PREPROCESSING SELESAI!


## 3. Lihat Detail Hasil Preprocessing (Opsional)

Melihat hasil setiap tahap preprocessing untuk beberapa contoh

In [10]:
# Lihat detail preprocessing untuk 3 contoh pertama
print("="*80)
print("DETAIL HASIL SETIAP TAHAP PREPROCESSING")
print("="*80)

for i in range(min(3, len(df_final))):
    print(f"\n{'='*80}")
    print(f"CONTOH {i+1}")
    print(f"{'='*80}")
    print(f"\n0. ASLI:")
    print(f"   {df_final['Teks_Komentar'].iloc[i][:100]}...")
    
    print(f"\n1. CLEANING:")
    print(f"   {df_final['teks_tahap1'].iloc[i][:100]}...")
    
    print(f"\n2. TOKENISASI:")
    tokens_2 = df_final['tokens_tahap2'].iloc[i]
    print(f"   {tokens_2[:10]}...")  # Tampilkan 10 token pertama
    print(f"   (Total: {len(tokens_2)} kata)")
    
    print(f"\n3. STOPWORD REMOVAL:")
    tokens_3 = df_final['tokens_tahap3'].iloc[i]
    print(f"   {tokens_3[:10]}...")  # Tampilkan 10 token pertama
    print(f"   (Total: {len(tokens_3)} kata)")
    
    print(f"\n4. STEMMING:")
    tokens_4 = df_final['tokens_tahap4'].iloc[i]
    print(f"   {tokens_4[:10]}...")  # Tampilkan 10 token pertama
    print(f"   (Total: {len(tokens_4)} kata)")
    
    print(f"\n5. FINAL (Gabungan):")
    print(f"   {df_final['teks_final'].iloc[i][:100]}...")
    print()

print("="*80)

DETAIL HASIL SETIAP TAHAP PREPROCESSING

CONTOH 1

0. ASLI:
   Kalo kabur mau kemana ke Singapur ,emang di Singapur tinggal dimana rakyat Singapur aja nggak punya ...

1. CLEANING:
   kalo kabur mau kemana ke singapur emang di singapur tinggal dimana rakyat singapur aja nggak punya r...

2. TOKENISASI:
   ['kalo', 'kabur', 'mau', 'kemana', 'singapur', 'emang', 'singapur', 'tinggal', 'dimana', 'rakyat']...
   (Total: 26 kata)

3. STOPWORD REMOVAL:
   ['kabur', 'kemana', 'singapur', 'emang', 'singapur', 'tinggal', 'dimana', 'rakyat', 'singapur', 'aja']...
   (Total: 19 kata)

4. STEMMING:
   ['kabur', 'mana', 'singapur', 'emang', 'singapur', 'tinggal', 'mana', 'rakyat', 'singapur', 'aja']...
   (Total: 19 kata)

5. FINAL (Gabungan):
   kabur mana singapur emang singapur tinggal mana rakyat singapur aja nggak rumah nyicil face tears jo...


CONTOH 2

0. ASLI:
   Klo sudah gelap susah terangnya lebih baik bubar...

1. CLEANING:
   klo sudah gelap susah terangnya lebih baik bubar...

2. TOK

## 4. Statistik Preprocessing

Menampilkan statistik lengkap dari proses preprocessing

In [11]:
# Hitung statistik lengkap
total_awal = len(df)
total_setelah_filter = len(df_filtered)
total_final = len(df_final)
persentase_valid = (total_final / total_awal * 100) if total_awal > 0 else 0

# Hitung rata-rata panjang teks
avg_len_asli = df_final['Teks_Komentar'].str.len().mean()
avg_len_final = df_final['teks_final'].str.len().mean()

# Hitung rata-rata jumlah kata
avg_words_asli = df_final['Teks_Komentar'].str.split().str.len().mean()
avg_words_final = df_final['teks_final'].str.split().str.len().mean()

# Hitung statistik tokens
avg_tokens_setelah_tokenisasi = df_final['tokens_tahap2'].apply(len).mean()
avg_tokens_setelah_stopword = df_final['tokens_tahap3'].apply(len).mean()
avg_tokens_setelah_stemming = df_final['tokens_tahap4'].apply(len).mean()

print("=" * 70)
print("📊 STATISTIK PREPROCESSING LENGKAP")
print("=" * 70)

print(f"\n📈 JUMLAH DATA:")
print(f"   {'Data awal':<30} : {total_awal:>10,} komentar")
print(f"   {'Setelah filter konteks':<30} : {total_setelah_filter:>10,} komentar")
print(f"   {'Data akhir valid':<30} : {total_final:>10,} komentar")
print(f"   {'Persentase data valid':<30} : {persentase_valid:>10.2f}%")
print(f"   {'Data terbuang':<30} : {total_awal - total_final:>10,} komentar")

print(f"\n📝 RATA-RATA PANJANG TEKS (karakter):")
print(f"   {'Teks asli':<30} : {avg_len_asli:>10.2f}")
print(f"   {'Teks final':<30} : {avg_len_final:>10.2f}")
print(f"   {'Reduksi':<30} : {((avg_len_asli - avg_len_final) / avg_len_asli * 100):>10.2f}%")

print(f"\n📄 RATA-RATA JUMLAH KATA:")
print(f"   {'Teks asli':<30} : {avg_words_asli:>10.2f} kata")
print(f"   {'Teks final':<30} : {avg_words_final:>10.2f} kata")
print(f"   {'Reduksi':<30} : {((avg_words_asli - avg_words_final) / avg_words_asli * 100):>10.2f}%")

print(f"\n🔄 RATA-RATA TOKENS PER TAHAP:")
print(f"   {'Setelah Tokenisasi':<30} : {avg_tokens_setelah_tokenisasi:>10.2f} tokens")
print(f"   {'Setelah Stopword Removal':<30} : {avg_tokens_setelah_stopword:>10.2f} tokens")
print(f"   {'Setelah Stemming':<30} : {avg_tokens_setelah_stemming:>10.2f} tokens")
print(f"   {'Reduksi total tokens':<30} : {((avg_tokens_setelah_tokenisasi - avg_tokens_setelah_stemming) / avg_tokens_setelah_tokenisasi * 100):>10.2f}%")

print("\n" + "=" * 70)
print("✅ Preprocessing berhasil!")
print(f"💾 File tersimpan: {output_file}")
print("=" * 70)

📊 STATISTIK PREPROCESSING LENGKAP

📈 JUMLAH DATA:
   Data awal                      :      2,196 komentar
   Setelah filter konteks         :        641 komentar
   Data akhir valid               :        641 komentar
   Persentase data valid          :      29.19%
   Data terbuang                  :      1,555 komentar

📝 RATA-RATA PANJANG TEKS (karakter):
   Teks asli                      :     209.97
   Teks final                     :     118.22
   Reduksi                        :      43.70%

📄 RATA-RATA JUMLAH KATA:
   Teks asli                      :      32.66 kata
   Teks final                     :      18.47 kata
   Reduksi                        :      43.43%

🔄 RATA-RATA TOKENS PER TAHAP:
   Setelah Tokenisasi             :      31.22 tokens
   Setelah Stopword Removal       :      18.47 tokens
   Setelah Stemming               :      18.47 tokens
   Reduksi total tokens           :      40.83%

✅ Preprocessing berhasil!
💾 File tersimpan: data_preprocessed.csv
